In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob
import sys
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
import math
from sklearn.model_selection import train_test_split
import os
import PIL
import tensorflow.keras.backend as K
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from save_weights_every_epoch import CallbackForSavingModelWeights
from multiprocessing import Pool
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score, recall_score, accuracy_score
import tensorflow_datasets as tfds
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib import pyplot
import multiprocessing as mp
sys.path.append('../scripts/')
from find_bad_ones import find_bad_ones
import pickle

/home/ubuntu/test_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
allowed_gpus = [x for x in range(8)]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [4]:
print(REPLICAS)

8


In [5]:
load_dotenv('../config_files/dev.env')

True

In [6]:
with open('./data.pickle', 'rb') as handle:
    updated_train_csv = pickle.load(handle)

def split_datasets(data, test_size = 0.01):
    train, test = train_test_split(data, test_size = test_size, random_state = 42) 
    train, val = train_test_split(train, test_size = test_size, random_state = 42)
    return train, val, test

In [7]:
train, val, test = split_datasets(updated_train_csv)

train_labels = train['class_labels'].values.tolist()
computed = compute_class_weight(class_weight='balanced', classes=[0, 1], y=train_labels)
class_weight_dict = {
    0: computed[0],
    1: computed[1]
}

In [8]:
def read_train_imgs(img, label, shape):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = shape)
    img = img / 255
    return img, label

def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, labels = data['fixed_paths'].values.tolist(), data['class_labels'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, labels, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(8048 * 1)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_train_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [9]:
def create_model(model_name, shape):
    with strategy.scope():
        input_layer = tf.keras.Input(shape = shape)
        construct = getattr(keras.applications, model_name)
        mid_layer = construct(include_top = False, 
                            weights = None, 
                            pooling = 'avg')(input_layer)
        last_layer = keras.layers.Dense(1, activation = 'sigmoid')(mid_layer)
        model = keras.Model(input_layer, last_layer)
    return model
def compile_new_model(model):
    with strategy.scope():
        loss = keras.losses.BinaryCrossentropy(label_smoothing=0.05)
        optimizer = keras.optimizers.SGD()
        prec = keras.metrics.Precision(name = 'prec')
        rec = keras.metrics.Recall(name = 'rec')
        model.compile(
            loss = loss,
            optimizer = optimizer,
            metrics = [prec, rec]
        )
    return model

In [10]:
os.environ['tb_path']

'/home/ubuntu/ship_segmentation/TB/'

In [15]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 128
train_dataset = get_data(train, shape=(32, 32), batch_size = batch_size)
val_dataset = get_data(val, shape=(32, 32), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (32, 32, 3))
model = compile_new_model(model)
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
184/184 [==============================] - 185s 259ms/step - loss: 0.6816 - prec: 0.3246 - rec: 0.1562 - val_loss: 0.5668 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00
Epoch 2/30
184/184 [==============================] - 39s 210ms/step - loss: 0.5519 - prec: 0.4758 - rec: 0.2385 - val_loss: 0.5562 - val_prec: 0.5000 - val_rec: 0.0023
Epoch 3/30
184/184 [==============================] - 38s 209ms/step - loss: 0.5122 - prec: 0.5281 - rec: 0.2991 - val_loss: 0.5565 - val_prec: 0.6588 - val_rec: 0.1299
Epoch 4/30
184/184 [==============================] - 38s 208ms/step - loss: 0.4811 - prec: 0.5765 - rec: 0.3441 - val_loss: 0.4640 - val_prec: 0.5734 - val_rec: 0.3898
Epoch 5/30
184/184 [==============================] - 38s 206ms/step - loss: 0.4670 - prec: 0.5966 - rec: 0.3887 - val_loss: 0.4673 - val_pre

In [16]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline_v1/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline/'
weights_save = CallbackForSavingModelWeights(weights_path, epoch_number=31)
batch_size = 128
train_dataset = get_data(train, shape=(32, 32), batch_size = batch_size)
val_dataset = get_data(val, shape=(32, 32), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (32, 32, 3))
model = compile_new_model(model)
model.load_weights('/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline/30.h5')
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
184/184 [==============================] - 172s 242ms/step - loss: 0.3373 - prec: 0.7850 - rec: 0.6445 - val_loss: 0.4081 - val_prec: 0.6700 - val_rec: 0.6311
Epoch 2/30
184/184 [==============================] - 39s 210ms/step - loss: 0.3330 - prec: 0.7888 - rec: 0.6492 - val_loss: 0.3788 - val_prec: 0.7194 - val_rec: 0.6009
Epoch 3/30
184/184 [==============================] - 38s 207ms/step - loss: 0.3316 - prec: 0.7898 - rec: 0.6536 - val_loss: 0.3745 - val_prec: 0.7683 - val_rec: 0.6079
Epoch 4/30
184/184 [==============================] - 38s 208ms/step - loss: 0.3293 - prec: 0.7972 - rec: 0.6552 - val_loss: 0.3774 - val_prec: 0.7205 - val_rec: 0.6520
Epoch 5/30
184/184 [==============================] - 38s 208ms/step - loss: 0.3255 - prec: 0.7992 - rec: 0.6640 - val_loss: 0.4080 - val_prec: 0.787

In [18]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline_64/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_64/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 128
train_dataset = get_data(train, shape=(64, 64), batch_size = batch_size)
val_dataset = get_data(val, shape=(64, 64), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (64, 64, 3))
model = compile_new_model(model)
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
184/184 [==============================] - 184s 261ms/step - loss: 0.6489 - prec: 0.3073 - rec: 0.0972 - val_loss: 0.5703 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00
Epoch 2/30
184/184 [==============================] - 41s 225ms/step - loss: 0.5496 - prec: 0.4712 - rec: 0.1734 - val_loss: 0.5416 - val_prec: 0.2273 - val_rec: 0.0232
Epoch 3/30
184/184 [==============================] - 42s 227ms/step - loss: 0.5069 - prec: 0.5339 - rec: 0.2608 - val_loss: 0.5957 - val_prec: 0.4400 - val_rec: 0.0255
Epoch 4/30
184/184 [==============================] - 42s 228ms/step - loss: 0.4808 - prec: 0.5815 - rec: 0.3388 - val_loss: 0.6401 - val_prec: 0.6087 - val_rec: 0.0650
Epoch 5/30
184/184 [==============================] - 42s 226ms/step - loss: 0.4645 - prec: 0.6023 - rec: 0.3890 - val_loss: 0.6708 - val_pre

In [19]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline_64_v1/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_64/'
weights_save = CallbackForSavingModelWeights(weights_path, epoch_number=31)
batch_size = 128
train_dataset = get_data(train, shape=(64, 64), batch_size = batch_size)
val_dataset = get_data(val, shape=(64, 64), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (64, 64, 3))
model = compile_new_model(model)
model.load_weights('/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_64/30.h5')
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
184/184 [==============================] - 187s 260ms/step - loss: 0.3133 - prec: 0.8231 - rec: 0.6829 - val_loss: 0.3358 - val_prec: 0.7760 - val_rec: 0.6752
Epoch 2/30
184/184 [==============================] - 42s 230ms/step - loss: 0.3115 - prec: 0.8268 - rec: 0.6867 - val_loss: 0.3180 - val_prec: 0.8254 - val_rec: 0.6473
Epoch 3/30
184/184 [==============================] - 42s 229ms/step - loss: 0.3085 - prec: 0.8269 - rec: 0.6893 - val_loss: 0.4022 - val_prec: 0.7473 - val_rec: 0.6311
Epoch 4/30
184/184 [==============================] - 42s 229ms/step - loss: 0.3062 - prec: 0.8300 - rec: 0.6961 - val_loss: 0.3267 - val_prec: 0.8424 - val_rec: 0.6450
Epoch 5/30
184/184 [==============================] - 42s 227ms/step - loss: 0.3042 - prec: 0.8327 - rec: 0.7007 - val_loss: 0.3190 - val_prec: 0.7958 - val_rec: 0.6961
Epoch 6/30
184/184 [==============================] - 42s 229ms/ste

In [20]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline_128/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_128/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 128
train_dataset = get_data(train, shape=(128, 128), batch_size = batch_size)
val_dataset = get_data(val, shape=(128, 128), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (128, 128, 3))
model = compile_new_model(model)
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

Epoch 1/30
184/184 [==============================] - 187s 297ms/step - loss: 0.6092 - prec: 0.4112 - rec: 0.1381 - val_loss: 0.6192 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00
Epoch 2/30
184/184 [==============================] - 52s 283ms/step - loss: 0.5211 - prec: 0.5183 - rec: 0.2663 - val_loss: 0.5579 - val_prec: 0.5128 - val_rec: 0.0464
Epoch 3/30
184/184 [==============================] - 52s 283ms/step - loss: 0.4843 - prec: 0.5614 - rec: 0.3410 - val_loss: 0.6362 - val_prec: 0.4049 - val_rec: 0.6566
Epoch 4/30
184/184 [==============================] - 52s 282ms/step - loss: 0.4479 - prec: 0.6240 - rec: 0.4201 - val_loss: 0.5203 - val_prec: 0.6974 - val_rec: 0.2459
Epoch 5/30
184/184 [==============================] - 52s 283ms/step - loss: 0.4247 - prec: 0.6740 - rec: 0.4781 - val_loss: 0.5731 - val_prec: 0.5862 - val_rec: 0.4733
Epoch 6/30
184/184 [==============================] - 52s 282ms/step - loss: 0.3943 - prec: 0.7252 - rec: 0.5182 - val_loss: 0.4434 - val_prec: 0.

In [ ]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/res_50_baseline_128_v1/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_128/'
weights_save = CallbackForSavingModelWeights(weights_path, epoch_number = 31)
batch_size = 128
train_dataset = get_data(train, shape=(128, 128), batch_size = batch_size)
val_dataset = get_data(val, shape=(128, 128), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('ResNet50', (128, 128, 3))
model = compile_new_model(model)
model.load_weights('/home/ubuntu/ml-data-training/ship_seg_weights/classification/res_50_baseline_128/30.h5')
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

In [11]:
K.clear_session()
log_dir = f"{os.environ['tb_path']}classification/efb0_baseline_128/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'/home/ubuntu/ml-data-training/ship_seg_weights/classification/efb0_baseline_128/'
weights_save = CallbackForSavingModelWeights(weights_path, epoch_number = 31)
batch_size = 128
train_dataset = get_data(train, shape=(128, 128), batch_size = batch_size)
val_dataset = get_data(val, shape=(128, 128), repeat = False, shuffle = False, batch_size=batch_size)
model = create_model('EfficientNetB0', (128, 128, 3))
model = compile_new_model(model)
model_hist = model.fit(
    train_dataset,
    validation_data = val_dataset,
    verbose = 1,
    epochs = 30,
    steps_per_epoch = len(train) // (batch_size * REPLICAS),
    callbacks = [
        tensorboard_callback,
        weights_save
    ],
)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu